In [21]:
import pandas as pd
import statsmodels.api as sm

from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

from sklearn.neighbors import NearestNeighbors

In [22]:
# Find and remove highly correlated features
def remove_collinear_features(X: pd.DataFrame, threshold=0.95):
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print("to_drop", to_drop)
    return X.drop(to_drop, axis=1)


def calculate_vif(X: pd.DataFrame):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [
        variance_inflation_factor(X.values, i) for i in range(X.shape[1])
    ]
    return vif_data

In [23]:
df = pd.read_csv("./data/gold/panel_data_treated.csv")
df_weekly = pd.read_csv("./data/gold/panel_data_weekly_treated_sample.csv")
df_cross = pd.read_csv("./data/gold/cross_section.csv")

df["date"] = pd.to_datetime(df["date"])

C:\Users\caca_\AppData\Local\Temp\ipykernel_25660\3688949128.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/gold/panel_data_treated.csv")
C:\Users\caca_\AppData\Local\Temp\ipykernel_25660\3688949128.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weekly = pd.read_csv("./data/gold/panel_data_weekly_treated_sample.csv")


In [32]:
important_columns = ["mep_id", "meetings", "questions_log"]
country_columns = [c for c in df_cross.columns if "country" in c]
p_groups_columns = [c for c in df_cross.columns if "political_group" in c]

columns_to_keep = (
    important_columns + country_columns + p_groups_columns
)

membership_columns = [c for c in df_cross.columns if c not in columns_to_keep][1:]

columns_to_keep += membership_columns

In [34]:
# Run a PSM analysis
df_psm = df_cross.copy()[columns_to_keep]
# # Create treatment and control groups
df_psm["treatment"] = (df_psm["meetings"] > 0).astype(int)

# Prepare features for PSM
x_columns_psm = columns_to_keep[2:]

X_psm = df_psm[x_columns_psm].copy()
y_psm = df_psm["treatment"]

# Fit logistic regression for propensity scores
logit = sm.Logit(y_psm, sm.add_constant(X_psm))
logit_fit = logit.fit()

         Current function value: 0.319744
         Iterations: 35


C:\Users\caca_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [35]:
logit_fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              treatment   No. Observations:                 1353
Model:                          Logit   Df Residuals:                     1280
Method:                           MLE   Df Model:                           72
Date:                Wed, 19 Mar 2025   Pseudo R-squ.:                  0.5387
Time:                        22:57:09   Log-Likelihood:                -432.61
converged:                      False   LL-Null:                       -937.77
Covariance Type:            nonrobust   LLR p-value:                1.786e-165
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
questions_log                        -0.0379      0.107     -0.353      0.724      -0.248       0.173
country_0                            -0.7304      1.182     -0.618      0.537      -3.047       1.586
country_AUT                           0.4176      0.650      0.643      0.520      -0.856       1.691
country_BEL                           0.5596      0.572      0.978      0.328      -0.562       1.681
country_BGR                          -1.1728      0.667     -1.759      0.079      -2.479       0.134
country_CYP                          -1.9624      1.023     -1.919      0.055      -3.967       0.042
country_CZE                          -0.7217      0.547     -1.319      0.187      -1.794       0.351
country_DEU                           0.4810      0.432      1.113      0.266      -0.366       1.328
country_DNK                           0.3833      0.724      0.529      0.597      -1.036       1.802
country_ESP                          -0.3100      0.436     -0.710      0.478      -1.165       0.545
country_EST                          -1.0492      1.318     -0.796      0.426      -3.633       1.534
country_FIN                           0.9928      0.717      1.385      0.166      -0.412       2.398
country_GBR                          -4.1739      0.831     -5.023      0.000      -5.802      -2.545
country_GRC                          -1.7018      0.620     -2.746      0.006      -2.916      -0.487
country_HRV                          -1.8181      0.994     -1.828      0.068      -3.767       0.131
country_HUN                           0.9682      0.637      1.519      0.129      -0.281       2.217
country_IRL                          -0.2207      0.794     -0.278      0.781      -1.778       1.337
country_ITA                          -0.2072      0.357     -0.580      0.562      -0.907       0.493
country_LTU                          -2.2856      0.795     -2.873      0.004      -3.845      -0.727
country_LUX                           1.2151      1.127      1.079      0.281      -0.993       3.423
country_LVA                           0.0920      0.930      0.099      0.921      -1.730       1.914
country_MLT                           1.5615      1.502      1.040      0.298      -1.382       4.505
country_NLD                           0.0318      0.507      0.063      0.950      -0.962       1.025
country_POL                          -0.4253      0.431     -0.988      0.323      -1.269       0.419
country_PRT                          -0.3125      0.525     -0.596      0.551      -1.341       0.716
country_ROU                          -0.7270      0.559     -1.299      0.194      -1.824       0.370
country_SVK                          -0.4356      0.645     -0.675      0.500      -1.700       0.829
country_SVN                           0.3864      1.014      0.381      0.703      -1.601       2.373
country_SWE                           0.6583      0.601      1.095      0.274  

In [36]:
# Calculate propensity scores
df_psm["propensity_score"] = logit_fit.predict()

# Separate treatment and control
treatment = df_psm[df_psm["treatment"] == 1]
control = df_psm[df_psm["treatment"] == 0]

# Find nearest neighbors
nbrs = NearestNeighbors(n_neighbors=1).fit(control[["propensity_score"]])
distances, indices = nbrs.kneighbors(treatment[["propensity_score"]])

# Get matched control group
matched_control = control.iloc[indices.flatten()]

# Combine matched samples
matched_df_psm = pd.concat([treatment, matched_control])


# Check balance of covariates
def check_balance(df_psm, features, treatment_col="treatment"):
    balance_stats = []
    for feature in features:
        treated_mean = df_psm[df_psm[treatment_col] == 1][feature].mean()
        control_mean = df_psm[df_psm[treatment_col] == 0][feature].mean()
        std_diff = (treated_mean - control_mean) / np.sqrt(
            (
                df_psm[df_psm[treatment_col] == 1][feature].var()
                + df_psm[df_psm[treatment_col] == 0][feature].var()
            )
            / 2
        )
        balance_stats.append(
            {
                "Feature": feature,
                "Treated Mean": treated_mean,
                "Control Mean": control_mean,
                "Std Diff": std_diff,
            }
        )
    return pd.DataFrame(balance_stats)


# Print balance statistics
print("\nCovariate Balance After Matching:")
check_balance(matched_df_psm, x_columns_psm)


Covariate Balance After Matching:


C:\Users\caca_\AppData\Local\Temp\ipykernel_25660\3446567537.py:25: RuntimeWarning: invalid value encountered in scalar divide
  std_diff = (treated_mean - control_mean) / np.sqrt(


,Feature,Treated Mean,Control Mean,Std Diff
0,questions_log,3.536373,3.599083,-0.046258
1,country_0,0.976119,0.962687,0.077998
2,country_AUT,0.028358,0.002985,0.205211
3,country_BEL,0.035821,0.017910,0.110857
4,country_BGR,0.019403,0.043284,-0.137273
...,...,...,...,...
68,COMMITTEE_PARLIAMENTARY_TEMPORARY,0.365672,0.368657,-0.006188
69,DELEGATION_JOINT_COMMITTEE,0.428358,0.423881,0.009048
70,DELEGATION_PARLIAMENTARY,0.988060,0.979104,0.070462
71,DELEGATION_PARLIAMENTARY_ASSEMBLY,0.652239,0.623881,0.058992


# DID

In [37]:
# df_did = df[df["mep_id"].isin(matched_df_psm.index)]
df_did = df_weekly[df_weekly["mep_id"].isin(matched_df_psm.index)]

## DID regression

In [9]:
# Now, run a regression with fixed effects
from linearmodels import PanelOLS

In [50]:
# Set the index for panel data
df_panel = df_did.set_index(["mep_id", "date"])



x_columns_did = ['treatment', 'received_treatment_and_started' ] + columns_to_keep[3:]

# Prepare dependent and independent variables
Y = df_panel["quetions_log"]
X = df_panel[x_columns_did]

# X = X.drop(to_drop_colinear_columns, axis=1)

model = sm.OLS(Y, sm.add_constant(X))

results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           quetions_log   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     3.735
Date:                Wed, 19 Mar 2025   Prob (F-statistic):           8.54e-09
Time:                        23:06:07   Log-Likelihood:                 615.17
No. Observations:                1127   AIC:                            -1182.
Df Residuals:                    1103   BIC:                            -1062.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [18]:
# Run panel regression with entity (MEP) fixed effects
model = PanelOLS(Y, X, entity_effects=True)
results_fe = model.fit()

print(results_fe)

ValueError: exog does not have full column rank. If you wish to proceed with model estimation irrespective of the numerical accuracy of coefficient estimates, you can set check_rank=False.